--- Day 9: Sensor Boost ---

You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt. It must be the Ceres monitoring station!

In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest BOOST program - Basic Operation Of System Test.

While BOOST (your puzzle input) is capable of boosting your sensors, for tenuous safety reasons, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a complete Intcode computer.

Your existing Intcode computer is missing one key feature: it needs support for parameters in relative mode.

Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.

The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.

The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.

For example, given a relative base of 50, a relative mode parameter of -7 refers to memory address 50 + -7 = 43.

The relative base is modified with the relative base offset instruction:

    Opcode 9 adjusts the relative base by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.

For example, if the relative base is 2000, then after the instruction 109,19, the relative base would be 2019. If the next instruction were 204,-34, then the value at address 1985 would be output.

Your Intcode computer will also need a few other capabilities:

    The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value 0 and can be read or written like any other memory. (It is invalid to try to access memory at a negative address, though.)
    The computer should have support for large numbers. Some instructions near the beginning of the BOOST program will verify this capability.

Here are some example programs that use these features:

    109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99 takes no input and produces a copy of itself as output.
    1102,34915192,34915192,7,4,7,99,0 should output a 16-digit number.
    104,1125899906842624,99 should output the large number in the middle.

The BOOST program will ask for a single input; run it in test mode by providing it the value 1. It will perform a series of checks on each opcode, output any opcodes (and the associated parameter modes) that seem to be functioning incorrectly, and finally output a BOOST keycode.

Once your Intcode computer is fully functional, the BOOST program should report no malfunctioning opcodes when run in test mode; it should only output a single value, the BOOST keycode. What BOOST keycode does it produce?


In [1]:
from copy import deepcopy
from collections import deque

class IntcodeComputer(object):
    
    def __init__(self, program, mem_size = 2048):
        self.mem = deepcopy(program)
        self.mem += ['0'] * (mem_size - len(self.mem))
        self.i = 0
        self.rb = 0
        self.inputs = deque([])
        self.last_out = None
        self.done = False
        
    def process_instruction(self, instruction):
        instruction = instruction.zfill(2)
        opcode = instruction[-2:]
        param_modes = instruction[:-2]
        
        if opcode == '01':
            self._add(param_modes)
        elif opcode == '02':
            self._mult(param_modes)
        elif opcode == '03':
            self._input(param_modes)
        elif opcode == '04':
            return self._output(param_modes)
        elif opcode == '05':
            self._jump_if_true(param_modes)
        elif opcode == '06':
            self._jump_if_false(param_modes)
        elif opcode == '07':
            self._less_than(param_modes)
        elif opcode == '08':
            self._equals(param_modes)
        elif opcode == '09':
            self._update_rb(param_modes)
        elif opcode == '99':
            self.done = True
            return 0
        else:
            raise ValueError(f"Unknown opcode: '{opcode}' from instruction: '{instruction}'")
        
    def _get_param_val(self, param_pos, mode):
        if mode == '0':
            return int(self.mem[int(self.mem[self.i+param_pos])])
        elif mode == '1':
            return int(self.mem[self.i+param_pos])
        elif mode == '2':
            return int(self.mem[int(self.mem[self.i+param_pos]) + self.rb])
        else:
            raise ValueError(f'Unknown mode: {mode}')
        
    def _write_param(self, param_pos, mode, value):
        if mode == '0':
            self.mem[int(self.mem[self.i+param_pos])] = str(value)
        elif mode == '1':
            self.mem[self.i+param_pos] = str(value)
        elif mode == '2':
            self.mem[int(self.mem[self.i+param_pos]) + self.rb] = str(value)
        else:
            raise ValueError(f'Unknown mode: {mode}')

        
    def _add(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        self._write_param(3, param_modes[-3], a+b)
        self.i += 4
    
    def _mult(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        self._write_param(3, param_modes[-3], a*b)
        self.i += 4
    
    def _input(self, param_modes):
        param_modes = param_modes.zfill(1)
        self._write_param(1, param_modes[-1], self.inputs.popleft())
        self.i += 2
            
    def _output(self, param_modes):
        param_modes = param_modes.zfill(1)
        out = self._get_param_val(1, param_modes[-1])
        self.i += 2
        self.last_out = out
        return out
        
    def _jump_if_true(self, param_modes):
        param_modes = param_modes.zfill(2)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a != 0:
            self.i = b
        else:
            self.i += 3
            
    def _jump_if_false(self, param_modes):
        param_modes = param_modes.zfill(2)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a == 0:
            self.i = b
        else:
            self.i += 3  
            
    def _less_than(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a < b:
            self._write_param(3, param_modes[-3], 1)
        else:
            self._write_param(3, param_modes[-3], 0)
        self.i += 4
        
    def _equals(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a == b:
            self._write_param(3, param_modes[-3], 1)
        else:
            self._write_param(3, param_modes[-3], 0)
        self.i += 4
        
    def _update_rb(self, param_modes):
        param_modes = param_modes.zfill(1)
        a = self._get_param_val(1, param_modes[-1])
        self.rb += a
        self.i += 2
        
    def run(self):
        while self.i < len(self.mem):
            out = self.process_instruction(self.mem[self.i])
            if out is not None:
                return out
            
program = open('input/day9.txt').read().splitlines()[0].split(',')

In [2]:
# Part 1
cmp = IntcodeComputer(program)
cmp.inputs.append('1')
cmp.run()

3280416268

--- Part Two ---

You now have a complete Intcode computer.

Finally, you can lock on to the Ceres distress signal! You just need to boost your sensors using the BOOST program.

The program runs in sensor boost mode by providing the input instruction the value 2. Once run, it will boost the sensors automatically, but it might take a few seconds to complete the operation on slower hardware. In sensor boost mode, the program will output a single value: the coordinates of the distress signal.

Run the BOOST program in sensor boost mode. What are the coordinates of the distress signal?


In [3]:
# Part 2
cmp = IntcodeComputer(program)
cmp.inputs.append('2')
cmp.run()

80210